In [1]:
from clickhouse_driver import Client

In [19]:
database = 'test'
table = 'metro_entries'
# Подключение к ClickHouse
client = Client(host='localhost', database=database)

# Создание базы данных
client.execute('CREATE DATABASE IF NOT EXISTS metro_data')

# Создание таблицы
client.execute('''
CREATE TABLE IF NOT EXISTS metro_entries (
    station String,
    line String,
    year UInt16,
    quarter String,
    entries UInt32,
    exits UInt32,
    global_id UInt64
) ENGINE = MergeTree()
ORDER BY (station, line, year, quarter)
''')



query = f'''
SELECT count(*) 
FROM system.tables 
WHERE database = '{database}' AND name = '{table}'
'''

# Выполнение запроса
table_exists = client.execute(query)[0][0]

# Вывод результата
if table_exists:
    print(f"Таблица {table} существует в базе данных {database}.")
else:
    print(f"Таблица {table} не существует в базе данных {database}.")

Таблица metro_entries существует в базе данных test.


In [4]:
import pandas as pd

In [36]:
# Загрузка данных из CSV-файла с указанием правильного разделителя и пропуском первой строки
df = pd.read_csv('data.csv', sep=';', encoding='utf-8')

# Удаление лишнего столбца
df = df.drop(columns=['Unnamed: 7'])


# Преобразование названий столбцов для соответствия структуре таблицы
df.columns = df.columns = ["station", "line", "year", "quarter", "entries", "exits", "global_id"]
# df = df[df['station'] != 'Станция метрополитена']

print(df.head())

# Преобразование данных в список кортежей для вставки
data = [tuple(x) for x in df.to_numpy()]

# Вставка данных в таблицу с обработкой исключений
try:
    client.execute('''
    INSERT INTO metro_entries (station, line, year, quarter, entries, exits, global_id) VALUES
    ''', data)
except ValueError as ve:
    print(f"ValueError occurred: {ve}")
except TypeError as te:
    print(f"TypeError occurred: {te}")
except client.Error as ce:
    print(f"ClickHouse Error occurred: {ce}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

                 station                       line  year    quarter  \
0  Станция метрополитена                      Линия   Год    Квартал   
1                 Митино  Арбатско-Покровская линия  2021  I квартал   
2          Волоколамская  Арбатско-Покровская линия  2021  I квартал   
3               Строгино  Арбатско-Покровская линия  2021  I квартал   
4             Крылатское  Арбатско-Покровская линия  2021  I квартал   

            entries              exits   global_id  
0  Входы пассажиров  Выходы пассажиров   global_id  
1           1913498            1829031  1138975996  
2           1236714            1222309  1138975997  
3           1938816            1903731  1138975999  
4           1849616            1818208  1138976000  


AttributeError: 'Client' object has no attribute 'Error'

In [25]:
query = '''
SELECT count(*) FROM metro_entries
'''

clickhouse_record_count = client.execute(query)[0][0]

clickhouse_record_count

7422

In [26]:
# Подсчет количества записей в CSV-файле
csv_record_count = len(df)

# Сравнение количества записей
if csv_record_count == clickhouse_record_count:
    print(f"Количество записей в ClickHouse совпадает с количеством записей в CSV-файле - {csv_record_count}" )
else:
    print(f"Количество записей не совпадает. В CSV-файле: {csv_record_count}, в ClickHouse: {clickhouse_record_count}.")

Количество записей не совпадает. В CSV-файле: 3711, в ClickHouse: 7422.


In [45]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Загрузка данных из CSV-файла с указанием правильного разделителя и пропуском первой строки
df = pd.read_csv('data.csv', sep=';', encoding='utf-8')

# Удаление лишнего столбца
df = df.drop(columns=['global_id'])

# Преобразование названий столбцов для соответствия структуре таблицы
df.columns = ["station", "line", "year", "quarter", "entries", "exits"]

# Удаление строки с названиями станций и переиндексация
df = df.reset_index(drop=True)

# # Удаление первой строки, так как она содержит заголовки столбцов
# df = df.iloc[1:]

# Преобразование категориальных признаков в числовые с помощью one-hot encoding
df = pd.get_dummies(df, columns=['station', 'line', 'year', 'quarter'])

# Вычисление матрицы корреляций
corr_matrix = df.corr()

# Визуализация матрицы корреляций с использованием тепловой карты
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()




ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements